In [1]:
# example from tensorflow website
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
PROPORTION_TRAIN = 0.9

In [3]:
data = pd.read_csv('data/train.csv')

In [4]:
images = data.iloc[:,1:].values
images = images.astype(np.float)
images = np.multiply(images, 1.0 / 255.0)

num_images = images.shape[0]
num_train = int(np.ceil(PROPORTION_TRAIN * num_images))

In [5]:
labels_flat = data[[0]].values.ravel()
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

labels = dense_to_one_hot(labels_flat, 10)
labels = labels.astype(np.uint8)

In [6]:
print(labels)

[[0 1 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 1 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 1]]


In [7]:
train_images = images[:num_train]
train_labels = labels[:num_train]
validation_images = images[num_train:]
validation_labels = labels[num_train:]

In [8]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
# graph
# input
x = tf.placeholder(tf.float32, shape=[None, 784])
x_image = tf.reshape(x, [-1,28,28,1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# conv layer 1
W_conv1 = weight_variable([5, 5, 1, 32]) # 5x5 filters
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# conv layer 2
W_conv2 = weight_variable([5, 5, 32, 64]) # 5x5 filters
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# fully connected layer 1
W_fc1 = weight_variable([7 * 7 * 64, 1024]) # 7x7 b/c 2xmax_pool (down samples by 2 along each dimension)
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# drop out layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# fully connected layer 2
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# softmax output
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [10]:
index_in_epoch = 0
num_examples = train_images.shape[0]

# serve data by batches
def next_batch(batch_size):
    global train_images
    global train_labels
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

In [16]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
for i in range(5000):
  batch = next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f837c0a6ba8>>
Traceback (most recent call last):
  File "/home/svatasoiu/anaconda3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 178, in __del__
    self.close()
  File "/home/svatasoiu/anaconda3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1028, in close
    self._default_session.__exit__(None, None, None)
  File "/home/svatasoiu/anaconda3/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/home/svatasoiu/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3529, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


step 0, training accuracy 0.08
step 100, training accuracy 0.86
step 200, training accuracy 0.96
step 300, training accuracy 0.92
step 400, training accuracy 0.92
step 500, training accuracy 0.92
step 600, training accuracy 0.98
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.94
step 1000, training accuracy 0.96
step 1100, training accuracy 1
step 1200, training accuracy 0.94
step 1300, training accuracy 0.98
step 1400, training accuracy 1
step 1500, training accuracy 0.94
step 1600, training accuracy 1
step 1700, training accuracy 0.98
step 1800, training accuracy 0.98
step 1900, training accuracy 0.94
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 0.98
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, tr

In [17]:
num_validated = 0
total_correct = 0
validate_batch_size = 100
while num_validated < len(validation_labels):
    start = num_validated
    end = min(num_validated + validate_batch_size, len(validation_labels))
    acc = accuracy.eval(feed_dict={x: validation_images[start:end], 
                                   y_: validation_labels[start:end], 
                                   keep_prob: 1.0})
    total_correct += int(np.round(acc * validate_batch_size))
    num_validated = end
    
print("total accuracy: %g" % (total_correct/len(validation_labels)))

total accuracy: 0.990714
